# Разработка рекомендательной системы
В рекомендательных системах применяется целый ряд технологий. Можно выделить две большие группы.

## Cистемы на основе фильтрации содержимого (Content-based)
Исследуются свойства рекомендуемых объектов. Например, если пользователь системы Netflix смотрел много ковбойских фильмов, то ему порекомендуют фильм, отнесенный в базе к жанру «ковбойский». Или, если пользователю нравятся фильмы с определенными актерами, то ему порекомендуют фильмы с таким же актерским составом или близким к нему.<br>
Т.е. "похожести" оцениваются по признакам содержимого объектов.

***Недостатки:***<br>
Сильная зависимость от предметной области, полезность рекомендаций ограничена.
Подход более трудоёмкий. Например нужно подготовить вектора профиля объекта используя методы нахождения «похожих» документов с помощью разбиения на шинглы, вычисления минхэшей и LSH-хэширования и т д.

## Системы коллаборативной фильтрации (Collaborative Filtering)
Рекомендуют объекты на основе сходства между пользователями и (или) объектами. Пользователю рекомендуются объекты, которые предпочитают похожие на него пользователи.<br>
Т.е. для рекомендации используется история оценок как самого пользователя, так и других пользователей.
Это более универсальный подход по сравнению с `Content-based` фильтрацией, часто дает лучший результат.

Вместо того чтобы определять сходство объектов по их признакам, мы будем анализировать сходство оценок, поставленных этим объектам пользователям. То есть вместо вектора профиля объекта мы возьмем соответствующий ему столбец матрицы предпочтений. А вместо того чтобы с муками изобретать вектор профиля пользователя, представим пользователя его строкой в матрице предпочтений.<br>
Пользователи считаются похожими, если их векторы близки с точки зрения некоторой метрики, например расстояния `Жаккара` или `косинусного расстояния`. Тогда для выработки рекомендации пользователю `U` мы смотрим, какие пользователи больше всего похожи на `U` в описанном выше смысле, и рекомендуем те объекты, которым им нравятся.

В большинстве случаев алгоритмы коллаборативной фильтрации (`Collaborative Filtering`) показывают лучший результат, чем `Content-based` системы. <br>
В данной работе будут рассматриваться два типа `Collaborative Filtering`: <br>
1. *Memory-Based Collaborative Filtering*
2. *Model-Based Collaborative filtering*

***Недостатки:***<br>
`"Холодный старт"` - что рекомендовать новым пользователям, которые не оценили ни одного фильма или кому рекомендовать фильм, у которого ещё не оценок?

## Коллаборативная фильтрация (Collaborative Filtering). 

#### Матрицы предпочтений. 
Рекомендательные системы имеют дело с пользователями и объектами. В матрице предпочтений хранится известная
информация о том, насколько объект нравится пользователю. Обычно большинство ее элементов неизвестны, а задача состоит в том, чтобы рекомендовать объекты пользователям, предсказав значения неизвестных элементов на основе известных.<br>

Нам нужно как можно точнее предсказать оценки под знаком вопроса:
<center>
<img src="ratings_predict.png" width=70%>
</center>

#### Сходство строк и столбцов матрицы предпочтений. 
Алгоритмы коллаборативной фильтрации должны измерять сходство строк и (или) столбцов матрицы предпочтений. `Расстояние Жаккара` годится для этой цели, если матрица содержит только единицы и незаполненные элементы («не оценено»). Для матриц более общего вида применяется `косинусное расстояние`.<br>
Часто перед тем как вычислять косинусное расстояние, бывает полезно нормировать матрицу предпочтений путем вычитания среднего значения (по строке, по столбцу или комбинацию того и другого) из каждого элемента.

####  В чем заключается сложность предсказания значений неизвестных элементов?
Можно предположить, что даже если два фильма относятся к одному жанру, вероятно, найдется очень мало пользователей, посмотревших или оценивших оба. И аналогично, даже если двум пользователям нравится один
или несколько жанров, маловероятно, что они посмотрели одни и те же фильмы.<br>
Типичный пользователь оценивает лишь малое подмножество всех фильмов поэтотму матрица очень разреженная, т.е. преобладают пустые элементы матрицы предпочтений в строке пользователеей.<br>
Поэтому найти сходство между пользователями или объектами трудно, потому что у нас
мало информации о парах пользователь-объект в разреженной матрице предпочтений.

#### Точность предсказаний
Точность предсказаний неизвестных элементов зависит от множества факторов. 
В первую очередь для качественного обучения алгоритма нам необходим большой набор данных. Но с ростом объема данных увеличивается сложность вычислений и время вычислений. При маленньком наборе - точность прогнозирования плохая, так как алгоритм не может обучится точно находить взаимосвязи в данных.<br>
Так же сильно влияет выбор алгоритма. На разных наборах данных один и тот же алгоритм может работать с разной точностью. Поэтому одна из задач провести тесты разных алгоритмов, оценить качество прогнозов и выбрать наилучший.<br>
Для обучения алгоритмов мы будем разбивать датасет на две части: `тренировочную` - для обучени и `тестовую` - на которой мы будем оценивать качество прогнозирования алгоритмов.<br>
Для оценки качества предсказания мы будем использовать метрику RMSE (Root Mean Square Error, корень из средней квадратичной ошибки) который расчитывается по формуле:
$$ RMSE = \sqrt{\frac{1}{N}\sum (x_i - \hat{x}_i)^2} $$
Чем меньше RMSE тем точнее алгоритм прогнозирует неизвестные элементы.

#### Алгоритмы
Далее мы рассмотрим несколько алгоритмов и качество прогнозиррованиня с использованнием разных техник и алгоритмов, от самого простого - "наивный метод" до знаменитого Model-Based алгоритма матричной факторизации SVD (Singular Value Decomposition), популяризированного Саймоном Фанком во время премии Netflix.
"Наивный метод" - является самым простым для понимания и этот алгоритм мы реализуем сами исползуя библиоеки Python numpy, pandas, sklearn и math.
Алгоритмы семейства KNN и SVD сложны в реализации с "нуля" и поэтому мы будем использовать библиотеку SurPRISE (Simple Python RecommendatIon System Engine) - https://surpriselib.com
<pre>
Surprise was designed with the following purposes in mind:

Give users perfect control over their experiments. To this end, a strong emphasis is laid on documentation, which we have tried to make as clear and precise as possible by pointing out every detail of the algorithms.
Alleviate the pain of Dataset handling. Users can use both built-in datasets (Movielens, Jester), and their own custom datasets.
Provide various ready-to-use prediction algorithms such as baseline algorithms, neighborhood methods, matrix factorization-based ( SVD, PMF, SVD++, NMF), and many others. Also, various similarity measures (cosine, MSD, pearson…) are built-in.
Make it easy to implement new algorithm ideas.
Provide tools to evaluate, analyse and compare the algorithms’ performance. Cross-validation procedures can be run very easily using powerful CV iterators (inspired by scikit-learn excellent tools), as well as exhaustive search over a set of parameters.
</pre>

## Memory-Based Collaborative Filtering

### 1. "Наивный метод"
В данном методе мы не будем разделять набор данных на тестовую и обучающую выборки.
RMSE будет оценивать матрицу прогнозов по всей выборке с исходной матрицей предпочтений.

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
# Данные
data = {'userId': ['Masha','Ira', 'Ira', 'Ira', 'Anna', 'Anna', 'Anna', 'Vova', 'Vova', 'Vova', 'Vova', 'Inna', 'Inna', 'Ivan', 'Ivan', 'Ivan'],
        'movieId': ['Ant-man','Thor 1', 'Thor 2', 'Thor 3', 'Thor 1', 'Thor 2', 'Thor 3', 'Thor 1', 'Thor 2', 'Ant-man', 'Hulk', 'Thor 1', 'Ant-man', 'Thor 1', 'Thor 3', 'Hulk'],
        'rating': [3,4,5,4,5,5,5,4,1,5,3,1,5,5,5,4]
        }

#data = {'userId': ['Anna', 'Anna', 'Anna', 'Vova', 'Vova', 'Vova', 'Vova', 'Inna', 'Inna', 'Ivan', 'Ivan', 'Ivan'],
#        'movieId': ['Thor 1', 'Thor 2', 'Thor 3', 'Thor 1', 'Thor 2', 'Ant-man', 'Hulk', 'Thor 1', 'Ant-man', 'Thor 1', 'Thor 3', 'Hulk'],
#        'rating': [5,5,5,4,1,5,3,1,5,5,5,4]
#        }


df_ratings = pd.DataFrame(data)
df_ratings

,userId,movieId,rating
0,Masha,Ant-man,3
1,Ira,Thor 1,4
2,Ira,Thor 2,5
3,Ira,Thor 3,4
4,Anna,Thor 1,5
5,Anna,Thor 2,5
6,Anna,Thor 3,5
7,Vova,Thor 1,4
8,Vova,Thor 2,1
9,Vova,Ant-man,5


In [3]:
# выводим количество пользователей и фильмов
n_users = df_ratings['userId'].unique().shape[0]
n_items = df_ratings['movieId'].unique().shape[0]
print(f"Пользователей: {n_users}\nФильмов: {n_items}")
print('Уникальные рейтинги:', sorted(df_ratings['rating'].unique()))

Пользователей: 6
Фильмов: 5
Уникальные рейтинги: [1, 3, 4, 5]


### 1.1 User-Item Collaborative Filtering

"Похожим на вас пользователям нравится это ..."

In [4]:
# Строим матрицу предпочтений Пользователи (userId) к объектам (movieId) используя Pandas pivot()
df_user_item = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0) # Неизвестные оценки (NaN) - заполняем "0"

df_user_item = df_user_item.astype(int)
print("Матрица предпочтений. Пользователи (userId) к объектам (movieId):")
df_user_item

Матрица предпочтений. Пользователи (userId) к объектам (movieId):


movieId,Ant-man,Hulk,Thor 1,Thor 2,Thor 3
userId,,,,,
Anna,0,0,5,5,5
Inna,5,0,1,0,0
Ira,0,0,4,5,4
Ivan,0,4,5,0,5
Masha,3,0,0,0,0
Vova,5,3,4,1,0


### 1.2 Расчет наиболее и наименее похожих объектов по матрице предпочтений df_user_item

In [5]:
matrix = df_user_item.values # Из pandas dataframe получаем numpy матрицу (только значения)
print("Матрица предпочтений из датафрейма df_user_item:\n", matrix)

Матрица предпочтений из датафрейма df_user_item:
 [[0 0 5 5 5]
 [5 0 1 0 0]
 [0 0 4 5 4]
 [0 4 5 0 5]
 [3 0 0 0 0]
 [5 3 4 1 0]]


In [6]:
# Поиск нескольких пар. Кол-во указывается в аргументе функции Npair
def find_similar_and_dissimilar_objects_pair(matrix, Npair=1):
    '''
        matrix - матрица предпочтений пользователи (userId) к объектам (movieId) в формате numpy array
        Npair - целое число. Количество вывоодимых пар похожих пользователей
    '''
    # вычисляем косинусное расстояние между строками матрицы
    distances = cosine_distances(matrix)
    #print(distances)
    
    for i in range(Npair):
        # находим индексы наиболее и наименее похожих объектов (расписать по шагам)
        most_similar = tuple(np.where(distances == distances[distances.nonzero()].min())[0])
        least_similar = np.unravel_index(np.argmax(distances), distances.shape)
        
        print(
            f"Пара №{i+1}\
            \nНаиболее похожие объекты: {df_user_item.index[most_similar[0]]} и {df_user_item.index[most_similar[1]]} с расстоянием {distances[most_similar]}\
            \nНаименее похожие объекты: {df_user_item.index[least_similar[0]]} и {df_user_item.index[least_similar[1]]} с расстоянием {distances[least_similar]}"
        )
        # Обнуляем найденые элементы матрицы
        distances[most_similar[0],most_similar[1]]=0
        distances[most_similar[1],most_similar[0]]=0
        distances[least_similar[0],least_similar[1]]=0
        distances[least_similar[1],least_similar[0]]=0


In [7]:
find_similar_and_dissimilar_objects_pair(matrix, Npair=3)

Пара №1            
Наиболее похожие объекты: Anna и Ira с расстоянием 0.005865153227565645            
Наименее похожие объекты: Anna и Masha с расстоянием 1.0
Пара №2            
Наиболее похожие объекты: Inna и Masha с расстоянием 0.019419324309079777            
Наименее похожие объекты: Ira и Masha с расстоянием 1.0
Пара №3            
Наиболее похожие объекты: Inna и Vova с расстоянием 0.20360919724741988            
Наименее похожие объекты: Ivan и Masha с расстоянием 1.0


In [8]:
# Расчитывааем сходство между пользователями. 
# Для этого считаем косинусное расстояние для пользователей используя pairwise_distances()
user_similarity = pairwise_distances(df_user_item.values, metric='cosine')
print("Размер матрицы:", user_similarity.shape)

# Формируем датафрейм для наглядности
df_user_user = pd.DataFrame(data=user_similarity, 
                            index=df_user_item.index.values, 
                            columns=df_user_item.index.values,
                            )
print("Косинусное расстояние для пользователей:")
df_user_user

Размер матрицы: (6, 6)
Косинусное расстояние для пользователей:


,Anna,Inna,Ira,Ivan,Masha,Vova
Anna,0.000000,0.886772,0.005865,0.289331,1.000000,0.595774
Inna,0.886772,0.000000,0.896095,0.879299,0.019419,0.203609
Ira,0.005865,0.896095,0.000000,0.347845,1.000000,0.610510
Ivan,0.289331,0.879299,0.347845,0.000000,1.000000,0.448440
Masha,1.000000,0.019419,1.000000,1.000000,0.000000,0.299860
Vova,0.595774,0.203609,0.610510,0.448440,0.299860,0.000000


In [9]:
df_user_item

movieId,Ant-man,Hulk,Thor 1,Thor 2,Thor 3
userId,,,,,
Anna,0,0,5,5,5
Inna,5,0,1,0,0
Ira,0,0,4,5,4
Ivan,0,4,5,0,5
Masha,3,0,0,0,0
Vova,5,3,4,1,0


Из матрицы видно что для `Anna` два наиболее близких пользователя: `Ira (растояние 0.005865)` и `Ivan (растояние 0.289331)`

Самая простейшая рекомендательная система считает предсказанную оценку пользователя `u` фильму `i` по формуле:

$$ r_{u,i} = \frac{\sum_{u' \in U} r_{u',i}}{N} $$


где:
N — количество пользователей, похожих на пользователя u,<br>
U — множество из N похожих пользователей,<br>
u' — пользователь, похожий на пользователя u (из множества U),<br>
$ r_{u', i} $ — оценка пользователя u' фильму i,<br>
$ r_{u,i} $ — предсказанная оценка фильма i.<br>

Т.е. <br>
- для `Anna` и фильма `Hulk` прогнноз будет считаться `(0+4)/2 = 2`<br>
где: `0` - оценка этого фильма `Ira`, `4` - оценка этого фильма `Ivan`, `2` - кол-во близких пользователей `(Ira и Ivan)`
- для `Anna` и фильма `Ant-man` прогнноз будет считаться `(0+0)/2 = 0`<br>
где: `0` - оценка этого фильма `Ira`, `0` - оценка этого фильма `Ivan`, `2` - кол-во близких пользователей `(Ira и Ivan)`


In [10]:
# Функция прогнозированиня оценок пользователей
def naive_predict(top):
    '''
        top - кол-во близких пользователеей
    '''
    # Структура для хранения для каждого пользователя оценки фильмов top наиболее похожих на него пользователей:
    # top_similar_ratings[0][1] - оценки всех фильмов одного из наиболее похожих пользователей на пользователя с ид 0.
    # Здесь 1 - это не ид пользователя, а порядковый номер.
    # создаём матрицу с "0" размером (n_users  x top х n_items)
    top_similar_ratings = np.zeros((n_users, top, n_items))

    for i in range(n_users):
        # Для каждого пользователя необходимо получить наиболее похожих пользователей:
        # Нулевой элемент не подходит, т.к. на этом месте находится похожесть пользователя самого на себя
        top_sim_users = user_similarity[i].argsort()[1:top + 1]
        
        # берём только оценки из "обучающей" выборки 
        top_similar_ratings[i] = df_user_item.values[top_sim_users]

    pred = np.zeros((n_users, n_items)) # создаём матрицу с "0" размером n_users х n_items
    for i in range(n_users):
        # Вычисляем средний рейинг фильма для похожих пользователей
        pred[i] = top_similar_ratings[i].sum(axis=0) / top
        
    return pred

In [12]:
# Функция расчета метрики  RMSE
def rmse(actual, predicted):
    '''
        actual - актуальная матрица предпочтений 
        predicted - матрица с прогнозами
    '''
    # Оставим оценки, предсказанные алгоритмом
    # прредварительно, если есть NaN - заменяется на "0"
    predicted = np.nan_to_num(predicted)[actual.nonzero()].flatten()
    
    # Оставим оценки, которые реально поставил пользователь
    actual = np.nan_to_num(actual)[actual.nonzero()].flatten()
    
    # Считаем RMSE
    # Если squared=False - возвращает RMSE, если squared=True - возвращает MSE
    rmse = mean_squared_error(actual, predicted, squared=False)
    
    return rmse

In [14]:
# Прогноз c 2мя близкими пользователямми
naive_pred = naive_predict(3)

# Формируем датафрейм для наглядности
df_naive_pred = pd.DataFrame(data=naive_pred, 
                            index=df_user_item.index.values, 
                            columns=df_user_item.columns.values,
                            )
print("Прогнозные рейтинги:")
df_naive_pred

Прогнозные рейтинги:


,Ant-man,Hulk,Thor 1,Thor 2,Thor 3
Anna,1.666667,2.333333,4.333333,2.000000,3.000000
Inna,2.666667,2.333333,3.000000,0.333333,1.666667
Ira,1.666667,2.333333,4.666667,2.000000,3.333333
Ivan,1.666667,1.000000,4.333333,3.666667,3.000000
Masha,3.333333,1.000000,3.333333,2.000000,1.666667
Vova,2.666667,1.333333,2.000000,0.000000,1.666667


In [83]:
# Оценка точности прогнозирования. (низкая точность, приемлимый результат толжен быть меньше "1")
print('RMSE: ', rmse(df_user_item.values, df_naive_pred.values))

RMSE:  1.948557158514987


In [23]:
# Функция вывода рекомендаций похожих пользователей и рекомендованых фильмов с заданым рейттингом
def user_recomendation_rating(User, N=2, RATING=3):
    '''
        User - Имя ползователя из матрицы предпочтений (userId)
        N - целоое число. Кол-во похожих пользователей на User (по убываннию), 
            по которым будет выводиться рекомендация фильмов.
        RATING - целоое число. Учитывается оценки фильмов к выводу с рейтингом >= RATING
    '''

    # Прогнозируем рейтинги и формируем датафрейм с прогнозами
    df_naive_pred = pd.DataFrame(data=naive_predict(N), 
                                index=df_user_item.index.values, 
                                columns=df_user_item.columns.values,
                                )
    
    # Просмотренные фильмы пользователя User
    ViewedUser = df_ratings[df_ratings['userId']==User]['movieId'].values
    print(f"Пользователь {User} посмотрел фильмы: \n{ViewedUser}")

    # Похожие пользователи
    SimilarUser = df_user_user[User].sort_values()[1:N+1].index.to_list()
    
    print(f"\nПохожие пользователи {SimilarUser} рекомендуют фильмы:")
    for i in df_naive_pred.loc[User, ~df_naive_pred.columns.isin(ViewedUser)].index:
        if df_naive_pred.loc[User,i] >= RATING:
            print(f'"{i}" | Прогнозный рейтинг:{df_naive_pred.loc[User,i]}')

In [24]:
user_recomendation_rating('Anna', N=2, RATING=2)

Пользователь Anna посмотрел фильмы: 
['Thor 1' 'Thor 2' 'Thor 3']

Похожие пользователи ['Ira', 'Ivan'] рекомендуют фильмы:
"Hulk" | Прогнозный рейтинг:2.0


###  1.3 Item-Item Collaborative Filtering: 
"Пользователям, которым нравится данный фильм, может так же понравиться это ..."

In [93]:
# Транспонируем матрицу (разворачиваем) и делаем movieId x userId
df_item_user = df_user_item.T
print("Матрица предпочтений. Объекты (movieId) к пользователям (userId):")
df_item_user

Матрица предпочтений. Объекты (movieId) к пользователям (userId):


userId,Anna,Inna,Ira,Ivan,Masha,Vova
movieId,,,,,,
Ant-man,0,5,0,0,3,5
Hulk,0,0,0,4,0,3
Thor 1,5,1,4,5,0,4
Thor 2,5,0,5,0,0,1
Thor 3,5,0,4,5,0,0


In [95]:
# Расчитывааем сходство между фильмами (Item). 
# Для этого считаем косинусное расстояние для фильмов используя pairwise_distances()
item_similarity = pairwise_distances(df_item_user.values, metric='cosine')
print("Размер матрицы:", item_similarity.shape)

# Формируем датафрейм для наглядности
df_item_item = pd.DataFrame(data=item_similarity, 
                            index=df_item_user.index.values, 
                            columns=df_item_user.index.values,
                            )
df_item_item

Размер матрицы: (5, 5)


,Ant-man,Hulk,Thor 1,Thor 2,Thor 3
Ant-man,0.000000,0.609433,0.642748,0.908850,1.000000
Hulk,0.609433,0.000000,0.297509,0.915983,0.507634
Thor 1,0.642748,0.297509,0.000000,0.246867,0.108271
Thor 2,0.908850,0.915983,0.246867,0.000000,0.224368
Thor 3,1.000000,0.507634,0.108271,0.224368,0.000000


Для объектов (Item) можно не прогнозировать рейтинги, так как поиск ближайших объектов осуществляется по косинусному растояннию из матрицы df_item_item.
Тут мы прогнозируем только для оценки качества алгоритма с помощью RMSE.

In [96]:
# Функция прогнозированиня для Item
def naive_predict_item(top):
    '''
        top - кол-во близких пользователеей
    '''
    top_similar_ratings = np.zeros((n_items, top, n_users))

    for i in range(n_items):
        top_sim_movies = item_similarity[i].argsort()[1:top + 1]
        top_similar_ratings[i] = df_item_user.values[top_sim_movies]
        
    pred = np.zeros((n_items, n_users))
    for i in range(n_items):
        pred[i] = top_similar_ratings[i].sum(axis=0) / top
    
    return pred

# Прогноз
naive_pred_film = naive_predict_item(2)
# Формируем датафрейм для наглядности
df_naive_pred_film = pd.DataFrame(data=naive_pred_film, 
                            index=df_item_user.index.values, 
                            columns=df_item_user.columns.values,
                            )
print("Прогнозные рейтинги фильмов:")
print(df_naive_pred_film)

print('\nRMSE: ', rmse(df_item_user.values, df_naive_pred_film.values))

Прогнозные рейтинги фильмов:
         Anna  Inna  Ira  Ivan  Masha  Vova
Ant-man   2.5   0.5  2.0   4.5    0.0   3.5
Hulk      5.0   0.5  4.0   5.0    0.0   2.0
Thor 1    5.0   0.0  4.5   2.5    0.0   0.5
Thor 2    5.0   0.5  4.0   5.0    0.0   2.0
Thor 3    5.0   0.5  4.5   2.5    0.0   2.5

RMSE:  1.964529205687714


In [97]:
# Функция вывода рекомендаций по похожим фильмам
def item_recomendation(Film,N=1):
    # N - кол-во подходящих фильмов по убываннию
    '''
        Film - Название фильма из матрицы предпочтений (movieId)
        N - целоое число. Кол-во выводимых фильмов похожих на Film (по убываннию).
    '''
    
    # Похожий фильм
    SimilarFilm = df_item_item[Film].sort_values()[1:N+1].index
    print(f'Пользователям, которым нравится фильм "{Film}", может так же понравиться:\n')
    for i in SimilarFilm:
        print(f'"{i}"')

In [98]:
item_recomendation('Thor 1', N=3)

Пользователям, которым нравится фильм "Thor 1", может так же понравиться:

"Thor 3"
"Thor 2"
"Hulk"


## 2. Коллаборативная фильтрация (Collaborative Filtering) на реальных данных. 

### "Наивный метод"

### Датасет
В данной работе используется `MovieLens Dataset (Small)`. Посмотреть информацию или скачать датасет можно [отсюда](https://grouplens.org/datasets/movielens/).

#### Описание данных
- `links.csv` $-$ связь между `id` фильма в датасете и `id` соответствующего фильма на `imdb.com` и `themoviedb.org`;
- `movies.csv` $-$ описание каждого фильма с его названием и жанрами;
- `ratings.csv` $-$ оценки пользователей фильмов с временной отметкой;
- `tags.csv` $-$ список тегов, которые поставил пользователь фильму, с временной отметкой.

Для данной задачи нам понадобятся только часть данных - информация о том, какой рейтинг ставили пользователи фильмам `ratings.csv` и названия фильмов из `movies.csv`.

In [25]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances
from sklearn.metrics import mean_squared_error
from math import sqrt

In [21]:
# Для скачивания датасета - раскоментировать.
#!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
#!unzip ml-latest-small.zip

In [28]:
# загружаем данные в Pandas датасет
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')
df_ratings = df_ratings.drop(columns=['timestamp']) #Удаляем лишнинй столбец 'timestamp'
df_movies = pd.read_csv('ml-latest-small/movies.csv')

# смотрим на структуру
print("Размерность матрицы df_ratings:", df_ratings.shape)
df_ratings.head()

Размерность матрицы df_ratings: (100836, 3)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [29]:
print("Размерность матрицы df_movies:", df_movies.shape)
df_movies.head()

Размерность матрицы df_movies: (9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [40]:
# выводим количество пользователей и фильмов
n_users = df_ratings['userId'].unique().shape[0]
n_items = df_ratings['movieId'].unique().shape[0]
print(f"Пользователей: {n_users}\nФильмов: {n_items}")
print('Уникальные рейтинги:', sorted(df_ratings['rating'].unique()))

Пользователей: 610
Фильмов: 9724
Уникальные рейтинги: [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


### 2.1 Строим матрицу предпочтений пользователей

Пользователи (userId) к объектам (movieId) используя Pandas pivot()

In [41]:
df_user_item = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0) # Неизвестные оценки (NaN) - заполняем "0"

print("Размерность матрицы", df_user_item.shape)
print("Матрица предпочтений. Пользователи (userId) к объектам (movieId):")
df_user_item.head()

Размерность матрицы (610, 9724)
Матрица предпочтений. Пользователи (userId) к объектам (movieId):


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# Фуннкция поиска нескольких ближайших пар. Кол-во указывается в аргументе функции Npair
def find_similar_and_dissimilar_objects_pair(matrix, Npair=1):
    '''
        matrix - матрица предпочтений пользователи (userId) к объектам (movieId) в формате numpy array
        Npair - целое число. Количество вывоодимых пар похожих пользователей
    '''
    # вычисляем косинусное расстояние между строками матрицы
    distances = cosine_distances(matrix)
    #print(distances)
    
    for i in range(Npair):
        # находим индексы наиболее и наименее похожих объектов
        most_similar = tuple(np.where(distances == distances[distances.nonzero()].min())[0])
        least_similar = np.unravel_index(np.argmax(distances), distances.shape)
        
        print(
            f"Пара №{i+1}\
            \nНаиболее похожие объекты: {df_user_item.index[most_similar[0]]} и {df_user_item.index[most_similar[1]]} с расстоянием {distances[most_similar]}\
            \nНаименее похожие объекты: {df_user_item.index[least_similar[0]]} и {df_user_item.index[least_similar[1]]} с расстоянием {distances[least_similar]}"
        )
        # Обнуляем найденые элементы матрицы
        distances[most_similar[0],most_similar[1]]=0
        distances[most_similar[1],most_similar[0]]=0
        distances[least_similar[0],least_similar[1]]=0
        distances[least_similar[1],least_similar[0]]=0


In [44]:
# Ищем наиболее и наименее похожих пользователей используя функцию из п1.3
find_similar_and_dissimilar_objects_pair(df_user_item.values, 5)

Пара №1            
Наиболее похожие объекты: 126 и 379 с расстоянием 0.18662719374650805            
Наименее похожие объекты: 1 и 175 с расстоянием 1.0
Пара №2            
Наиболее похожие объекты: 130 и 574 с расстоянием 0.19995448500490587            
Наименее похожие объекты: 1 и 306 с расстоянием 1.0
Пара №3            
Наиболее похожие объекты: 130 и 468 с расстоянием 0.22310246169524028            
Наименее похожие объекты: 1 и 397 с расстоянием 1.0
Пара №4            
Наиболее похожие объекты: 242 и 468 с расстоянием 0.2387557518192306            
Наименее похожие объекты: 1 и 496 с расстоянием 1.0
Пара №5            
Наиболее похожие объекты: 150 и 270 с расстоянием 0.24768805448250675            
Наименее похожие объекты: 1 и 506 с расстоянием 1.0


### 2.2 Считаем косинусное расстояние для пользователей  (построчно).
Можно считать, что косинусное расстояние обозначает степень похожести.<br> 
Чем пользователи или фильмы более похожи друг на друга — тем меньше будет косинусное расстояние.

In [46]:
# Расчитывааем сходство между пользователями.
user_similarity = pairwise_distances(df_user_item.values, metric='cosine')
print("Размер матрицы user_similarity: ", user_similarity.shape)

Размер матрицы user_similarity:  (610, 610)


In [48]:
# То есть user_similarity[i][j] — косинусное расстояние между i-ой строкой и j-ой строкой 
# (можно проверить через scipy.spatial.distance.cosine(x,y) где x и y строки - это вектора), 
# а для item_similarity[i][j] — косинусное расстояние между i-ой и j-ой колонками.
print(user_similarity[125][378])

0.18662719374650805


In [49]:
# Прооверка косинусного растояния между векторами (строка 125 и 378 из матрицы df_user_item_train)
print(distance.cosine(df_user_item.values[125], df_user_item.values[378]))

0.18662719374650816


### 2.3 User-Item Collaborative Filtering: 
"Похожим на вас пользователям нравится это ..."

In [50]:
# Формируем датафрейм для наглядности из user_similarity - сходство между пользователями. 
df_user_user = pd.DataFrame(data=user_similarity, 
                            index=df_user_item.index.values, 
                            columns=df_user_item.index.values
                            )
print("Размер матрицы:", df_user_user.shape)
print("Матрица сходства между пользователями:")
df_user_user.head()

Размер матрицы: (610, 610)
Матрица сходства между пользователями:


,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,0.000000,0.972717,0.940280,0.805605,0.870920,0.871848,0.841256,0.863032,0.935737,0.983125,...,0.919446,0.835545,0.778514,0.929331,0.846375,0.835809,0.730611,0.708903,0.906428,0.854679
2,0.972717,0.000000,1.000000,0.996274,0.983386,0.974667,0.972415,0.972743,1.000000,0.932555,...,0.797329,0.983134,0.988003,1.000000,1.000000,0.971571,0.987052,0.953789,0.972435,0.897573
3,0.940280,1.000000,0.000000,0.997749,0.994980,0.996064,1.000000,0.995059,1.000000,1.000000,...,0.994952,0.995108,0.975008,1.000000,0.989306,0.987007,0.980753,0.978872,1.000000,0.967881
4,0.805605,0.996274,0.997749,0.000000,0.871341,0.911509,0.884880,0.937031,0.988639,0.968837,...,0.914062,0.871727,0.692027,0.947015,0.915416,0.799605,0.868254,0.850142,0.967802,0.892317
5,0.870920,0.983386,0.994980,0.871341,0.000000,0.699651,0.891658,0.570925,1.000000,0.969389,...,0.931952,0.581253,0.889852,0.741227,0.851242,0.893565,0.847134,0.864465,0.738768,0.939208


In [51]:
# Функция прогнозированиня оценок пользователей
def naive_predict(top):
    '''
        top - кол-во близких пользователеей
    '''
    # Структура для хранения для каждого пользователя оценки фильмов top наиболее похожих на него пользователей:
    # top_similar_ratings[0][1] - оценки всех фильмов одного из наиболее похожих пользователей на пользователя с ид 0.
    # Здесь 1 - это не ид пользователя, а просто порядковый номер.
    top_similar_ratings = np.zeros((n_users, top, n_items))
    #print(top_similar_ratings)

    for i in range(n_users):
        # Для каждого пользователя необходимо получить наиболее похожих пользователей:
        # Нулевой элемент не подходит, т.к. на этом месте находится похожесть пользователя самого на себя
        top_sim_users = user_similarity[i].argsort()[1:top + 1]
        #print(top_sim_users)
        
        # берём только оценки из "обучающей" выборки 
        top_similar_ratings[i] = df_user_item.values[top_sim_users]
        #print(top_similar_ratings)

    pred = np.zeros((n_users, n_items))
    for i in range(n_users):
        #print(top_similar_ratings[i])
        #print(top_similar_ratings[i].sum(axis=0))
        pred[i] = top_similar_ratings[i].sum(axis=0) / top
        
    return pred

In [55]:
# Строим прогноз
naive_pred = naive_predict(5)

# Формируем датафрейм для наглядности
df_naive_pred = pd.DataFrame(data=naive_pred, 
                            index=df_user_item.index.values, 
                            columns=df_user_item.columns.values,
                            )
print("Прогнозные рейтинги:")
df_naive_pred

Прогнозные рейтинги:


,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,2.2,0.6,1.2,0.0,0.0,3.8,0.0,0.0,0.0,2.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.8,0.0,0.0,0.0,2.4,0.0,0.0,0.0,0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,2.2,0.2,0.0,0.0,1.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.8,0.6,0.6,0.0,1.2,0.6,0.6,0.0,0.0,1.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.8,1.7,0.8,0.0,1.1,2.1,2.0,0.6,0.0,1.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,2.4,2.1,1.2,0.0,0.0,2.2,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.8,2.4,0.9,0.0,0.4,4.2,0.4,0.6,0.0,3.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
609,0.6,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,2.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
# Функция расчета метрики  RMSE
def rmse(actual, predicted):
    '''
        actual - актуальная матрица предпочтений 
        predicted - матрица с прогнозами
    '''
    # Оставим оценки, предсказанные алгоритмом
    predicted = np.nan_to_num(predicted)[actual.nonzero()].flatten()
    #print(predicted)
    
    # Оставим оценки, которые реально поставил пользователь
    actual = np.nan_to_num(actual)[actual.nonzero()].flatten()
    #print(actual)
    
    # Считаем RMSE
    # Если squared=False возвращает RMSE, если squared=True возвращает MSE
    rmse = mean_squared_error(actual, predicted, squared=False)
    return rmse

In [57]:
print('RMSE: ', rmse(df_user_item.values, df_naive_pred.values))

RMSE:  2.467566478745013


In [58]:
# Функция вывода рекомендаций по похожим пользователям с учетом рейтинга пользователей
def user_recomendation_rating(UserID, N=1, RATING=3):
    '''
        UserID - ID ползователя из матрицы предпочтений (userId)
        N - целоое число. Кол-во похожих пользователей на User (по убываннию), 
            по которым будет выводиться рекомендация фильмов.
        RATING - целоое число. Учитывается оценки фильмов к выводу с рейтингом >= RATING
    '''
    # Формируем датафрейм для наглядности
    df_naive_pred = pd.DataFrame(data=naive_predict(N), 
                                index=df_user_item.index.values, 
                                columns=df_user_item.columns.values,
                                )
    
    # Просмотренные фильмы пользователя UserID
    ViewedUser = df_ratings[df_ratings['userId']==UserID]['movieId'].values
    print(f"Пользователь {UserID} посмотрел фильмы:\n")
    for i in ViewedUser:
        res = df_movies[df_movies['movieId'] == i][['movieId','title','genres']].values
        print(f'Id:{res[0][0]}\t"{res[0][1]}"\t{res[0][2]}')

    # Похожии пользователи
    SimilarUser = df_user_user[UserID].sort_values()[1:N+1].index.to_list()
    
    print(f"\nПохожие пользователи {SimilarUser} рекомендуют фильмы:")
    for i in df_naive_pred.loc[UserID, ~df_naive_pred.columns.isin(ViewedUser)].index:
        if df_naive_pred.loc[UserID,i] >= RATING:
            res = df_movies[df_movies['movieId'] == i][['movieId','title','genres']].values
            print(f'Id:{res[0][0]}\t"{res[0][1]}"\t{res[0][2]} Прогнозный рейтинг:{df_naive_pred.loc[UserID,i]}')
    

In [63]:
user_recomendation_rating(UserID=126, N=5, RATING=3)

Пользователь 126 посмотрел фильмы:

Id:34	"Babe (1995)"	Children|Drama
Id:47	"Seven (a.k.a. Se7en) (1995)"	Mystery|Thriller
Id:110	"Braveheart (1995)"	Action|Drama|War
Id:150	"Apollo 13 (1995)"	Adventure|Drama|IMAX
Id:153	"Batman Forever (1995)"	Action|Adventure|Comedy|Crime
Id:161	"Crimson Tide (1995)"	Drama|Thriller|War
Id:165	"Die Hard: With a Vengeance (1995)"	Action|Crime|Thriller
Id:185	"Net, The (1995)"	Action|Crime|Thriller
Id:208	"Waterworld (1995)"	Action|Adventure|Sci-Fi
Id:231	"Dumb & Dumber (Dumb and Dumber) (1994)"	Adventure|Comedy
Id:253	"Interview with the Vampire: The Vampire Chronicles (1994)"	Drama|Horror
Id:288	"Natural Born Killers (1994)"	Action|Crime|Thriller
Id:292	"Outbreak (1995)"	Action|Drama|Sci-Fi|Thriller
Id:296	"Pulp Fiction (1994)"	Comedy|Crime|Drama|Thriller
Id:300	"Quiz Show (1994)"	Drama
Id:316	"Stargate (1994)"	Action|Adventure|Sci-Fi
Id:318	"Shawshank Redemption, The (1994)"	Crime|Drama
Id:329	"Star Trek: Generations (1994)"	Adventure|Drama|Sci-Fi
I

### 2.4 Item-Item Collaborative Filtering: 
"Пользователям, которым нравится данный фильм, может так же понравиться это ..."

In [64]:
# Транспонируем матрицу (разворачиваем) и делаем movieId x userId
df_item_user = df_user_item.T
print("Размер матрицы:", df_item_user.shape)
print("Матрица предпочтений. Объекты (movieId) к пользователям (userId):")
df_item_user.head()

Размер матрицы: (9724, 610)
Матрица предпочтений. Объекты (movieId) к пользователям (userId):


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


### Считаем косинусное расстояние для фильмов (поколоночно).

In [65]:
# Расчитывааем сходство между фильмами.
item_similarity = pairwise_distances(df_user_item.values.T, metric='cosine') # Транспонируем матрицу (.T)
print("Размер матрицы item_similarity: ", item_similarity.shape)
# Формируем датафрейм для наглядности из item_similarity - сходство фильмов
df_item_item = pd.DataFrame(data=item_similarity, 
                            index=df_user_item.T.index.values, 
                            columns=df_user_item.T.index.values
                            )
print("Матрица сходства между фильмами:")
df_item_item.head()

Размер матрицы item_similarity:  (9724, 9724)
Матрица сходства между фильмами:


,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,0.000000,0.589438,0.703083,0.964427,0.691238,0.623684,0.722509,0.868371,0.767414,0.604427,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.589438,0.000000,0.717562,0.893585,0.712205,0.702991,0.771424,0.827502,0.955165,0.582307,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.703083,0.717562,0.000000,0.907594,0.582198,0.715743,0.597169,0.686566,0.695160,0.757046,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.964427,0.893585,0.907594,0.000000,0.811624,0.910315,0.724965,0.841978,1.000000,0.904402,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,0.691238,0.712205,0.582198,0.811624,0.000000,0.701031,0.525998,0.716477,0.664942,0.781939,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Для объектов (Item) можно не прогнозировать рейтинги, так как поиск ближайших объектов осуществляется по косинусному растояннию из матрицы df_item_item.
Тут мы прогнозируем только для оценки качества алгоритма с помощью RMSE.

In [66]:
# Функция прогнозирования
def naive_predict_item(top):
    top_similar_ratings = np.zeros((n_items, top, n_users))
    #print(top_similar_ratings)

    for i in range(n_items):
        top_sim_movies = item_similarity[i].argsort()[1:top + 1]
        top_similar_ratings[i] = df_item_user.values[top_sim_movies]
        
    pred = np.zeros((n_items, n_users))
    for i in range(n_items):
        pred[i] = top_similar_ratings[i].sum(axis=0) / top
    
    return pred

# Прогноз
naive_pred_film = naive_predict_item(10)
# Формируем датафрейм для наглядности
df_naive_pred_film = pd.DataFrame(data=naive_pred_film, 
                            index=df_item_user.index.values, 
                            columns=df_item_user.columns.values,
                            )
print("Прогнозные рейтинги фильмов:")
print(df_naive_pred_film.head())
print('\nRMSE: ', rmse(df_item_user.values, df_naive_pred_film.values))

Прогнозные рейтинги фильмов:
   1    2    3    4    5    6     7    8    9     10   ...   601  602  603  \
1  3.3  0.0  0.0  1.2  0.3  2.0  3.70  1.2  0.5  0.35  ...  0.35  2.4  1.9   
2  1.1  0.0  0.0  0.7  1.6  3.5  1.15  2.0  0.0  0.40  ...  0.00  2.4  0.0   
3  0.8  0.0  0.0  0.4  0.0  3.0  0.00  0.3  0.0  0.00  ...  0.00  0.7  0.3   
4  0.0  0.0  0.0  0.0  0.0  2.1  0.00  0.0  0.0  0.00  ...  0.00  0.0  0.0   
5  1.2  0.0  0.0  0.4  0.0  3.8  0.00  0.0  0.0  0.00  ...  0.00  0.0  0.5   

   604  605   606  607   608  609   610  
1  0.4  2.7  2.40  2.3  3.15  0.7  3.60  
2  2.0  1.8  1.35  0.4  2.55  0.3  0.85  
3  1.0  0.0  0.60  0.5  1.95  0.0  0.35  
4  0.0  0.0  0.00  0.0  0.00  0.0  0.00  
5  1.2  0.4  0.25  0.5  1.50  0.0  0.00  

[5 rows x 610 columns]

RMSE:  2.0905672161059363


In [67]:
# Функция вывода рекомендаций по похожим фильмам
def item_recomendation(IDFilm, N=1):
    '''
        IDFilm - ID фильма из матрицы предпочтений (movieId)
        N - целоое число. Кол-во выводимых фильмов похожих на Film (по убываннию).
    '''
    
    FilmName = ' '.join(df_movies[df_movies['movieId']==IDFilm]['title'].values)
    # Похожий фильм
    SimilarFilm = df_item_item[IDFilm].sort_values()[1:N+1].index
    
    print(f'Пользователям, которым нравится фильм Id:{IDFilm} "{FilmName}", может так же понравиться:\n')
    for i in SimilarFilm:
        res = df_movies[df_movies['movieId'] == i][['movieId','title','genres']].values
        print(f'Id:{res[0][0]}\t"{res[0][1]}"\t{res[0][2]}')

In [69]:
item_recomendation(IDFilm=2628, N=10)

Пользователям, которым нравится фильм Id:2628 "Star Wars: Episode I - The Phantom Menace (1999)", может так же понравиться:

Id:1196	"Star Wars: Episode V - The Empire Strikes Back (1980)"	Action|Adventure|Sci-Fi
Id:1580	"Men in Black (a.k.a. MIB) (1997)"	Action|Comedy|Sci-Fi
Id:1210	"Star Wars: Episode VI - Return of the Jedi (1983)"	Action|Adventure|Sci-Fi
Id:260	"Star Wars: Episode IV - A New Hope (1977)"	Action|Adventure|Sci-Fi
Id:5378	"Star Wars: Episode II - Attack of the Clones (2002)"	Action|Adventure|Sci-Fi|IMAX
Id:2571	"Matrix, The (1999)"	Action|Sci-Fi|Thriller
Id:2115	"Indiana Jones and the Temple of Doom (1984)"	Action|Adventure|Fantasy
Id:3793	"X-Men (2000)"	Action|Adventure|Sci-Fi
Id:2716	"Ghostbusters (a.k.a. Ghost Busters) (1984)"	Action|Comedy|Sci-Fi
Id:5349	"Spider-Man (2002)"	Action|Adventure|Sci-Fi|Thriller


## Model-Based Collaborative filtering

### Прогнозирование с использованием библиотеки scikit-surprise
Документация: https://surprise.readthedocs.io/en/stable/

`scikit-surprise` (Simple Python Recommended System Engine) - это библлиотека машинного обучениня для создания и анализа рекомендательных систем, которые работают с явными рейтинговыми данными.

#### В `scikit-surprise` есть несколько семейств алгоритмов прогнозирования:
- Базовые алгоритмы
    - `NormalPredictor` - Алгоритм прогнозирования случайного рейтинга на основе распределения обучающей выборки, которое считается нормальным.
    - `BaselineOnly` - Алгоритм прогнозирования базовой оценки для данного пользователя и элемента.
    - `SlopeOne` - Простой, но точный алгоритм совместной фильтрации.
    - `CoClustering` - Алгоритм совместной фильтрации, основанный на совместной кластеризации.


- kNN (k Nearest Neighbor или k Ближайших Соседей) алгоритмы классификации
    - `KNNBasic` - Базовый алгоритм совместной фильтрации.
    - `KNNWithMeans` - Базовый алгоритм совместной фильтрации, учитывающий средние оценки каждого пользователя.
    - `KNNWithZScore` - Базовый алгоритм совместной фильтрации, учитывающий нормализацию z-оценки каждого пользователя.
    - `KNNBaseline` - Базовый алгоритм совместной фильтрации с учетом базовой оценки.
    
    Задача классификации в машинном обучении — это задача отнесения объекта к одному из заранее определенных классов на основании его формализованных признаков. Каждый из объектов в этой задаче представляется в виде вектора в N-мерном пространстве, каждое измерение в котором представляет собой описание одного из признаков объекта.<br>
    Алгортмы семейства kNN удобно использовать для получения k-ближайших соседей пользователя (или элемента). Можно использовать методы `get_neighbors()` объекта алгоритма. Это актуально только для алгоритмов, использующих меру сходства, таких как алгоритмы k-NN.


- Алгоритмы основанные на матричной факторизации
    - `SVD (Singular Value Decomposition - Сингулярное разложение)` - Знаменитый алгоритм SVD, популяризированный Саймоном Фанком во время премии Netflix.
    - `SVDpp` - Алгоритм SVD++, расширение SVD, учитывающее неявные рейтинги.
    - `NMF` - Алгоритм совместной фильтрации, основанный на неотрицательной матричной факторизации.

#### Модуль сходства включает в себя инструменты для вычисления показателей сходства между пользователями или элементами.
- `cosine` - Вычисляет косинусное растояние между всеми парами пользователей (или элементов).
- `msd` - Вычисляет среднеквадратичное растояние между всеми парами пользователей (или элементов).
- `pearson` - Вычисляет коэффициент корреляции Пирсона между всеми парами пользователей (или элементов).
- `pearson_baseline` - Вычисляет (сокращенный) коэффициент корреляции Пирсона между всеми парами пользователей (или элементов), используя базовые линии для центрирования вместо средних значений.

#### Модуль accuracy предоставляет инструменты для вычисления метрик точности набора прогнозов.
Доступные показатели точности:
- `rmse` - Вычисляет RMSE (среднеквадратичная ошибка).
- `mse` - Вычисляет MSE (среднеквадратическую ошибку).
- `mae` - Вычисляет MAE (средняя абсолютная ошибка).
- `fcp` - Вычисляет FCP (доля согласованных пар).

Так же у каждого алгоритма есть множество дополнительных настроек, которые позволяют добиться наилучших результатов прогнозирования (см. документацию). 

In [46]:
# Если библиоотека не установлена в Python, то раскомменитровать и установить.
#!pip install scikit-surprise

In [71]:
import io
import random
import time
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, accuracy
from surprise import SVD, SVDpp, BaselineOnly, NormalPredictor, SlopeOne, CoClustering, NMF, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import cross_validate, train_test_split
from collections import defaultdict

# Для воспроизводимых экспериментов
my_seed = 45
random.seed(my_seed)
np.random.seed(my_seed)

### Разделенине датасета на обучающщий и тестовый наборы данных.
Для тестирования и обучения алгоритмов в задачах `ML (machine learning)` исходный набор данных разделяют на два датасета: `обучающий (trainset)` и `тестовый (testset)` наборы данных в соотношенини 70-75% к 30-25%.

В библиотеке `scikit-surprise` уже реализована функция для разделения данных `train_test_split()`

На наборе `trainset` алгоритм сначала обучают (процедура `algo.fit()`), а на наборе `testset` алгоритм тестируют (`algo.test()`) на точность прогнозирования. Результатом `algo.test()` будут прогнозируемые метрики, которые сравнивают с оригинальными. 

Идея такого подхода заключается в том, что при обучении алгоритму дают известные данные с проставленными метриками (в нашем случае - рейтитнги фильмов). И на этих данных обучают модель.

При тестировании - из тестового набора, который алгоритм не "видел" до этого момента, убирают известные метрики (рейтинги) и обученный алгоритм должен их спрогнозировать. После чего сравнивают реальные метрики с прогнозными. Чем меньше разница между реальным и прогнозным значением - тем точнее алгоритм.

Для оценки точности мы будем использовать `RMSE` (среднеквадратичная ошибка).

In [72]:
# Reader - требуется указать параметр рейтинговой шкалы c min и max рейтинга в датасете.
# В нашем случае рейтинги  min: 0.5, max: 5.0
reader = Reader(rating_scale=(0.5, 5.0))

# Столбцы должны соответствовать user id, item id и ratings (в порядке как в датасете).
data = Dataset.load_from_df(df_ratings[["userId", "movieId", "rating"]], reader)

# Разделение датасета на обучающий (trainset) и тестовый (testset) наборы данных в 
# соотношенини 75% к 25% по кол-ву рейтингов. 
# Указывается только % тестового (test_size) или тренировочного (train_size) датасета
# Датасет перемешивается случайным образом перед разделеинем (shuffle=True).
trainset, testset = train_test_split(data, test_size=0.25, random_state=my_seed, shuffle=True)

print(f"Обучающий датасет trainset:\nКол-во пользователей:{trainset.n_users}\
\nКол-во рейтингов:{trainset.n_ratings}\nКол-во фильмов:{trainset.n_items}")

print(f"\nТестовый датасет testset:\nКол-во пользователей:{len(pd.DataFrame.from_records(testset)[0].unique())}\
\nКол-во рейтингов:{len(pd.DataFrame.from_records(testset)[2])}\
\nКол-во фильмов:{len(pd.DataFrame.from_records(testset)[1].unique())}")

Обучающий датасет trainset:
Кол-во пользователей:610
Кол-во рейтингов:75627
Кол-во фильмов:8781

Тестовый датасет testset:
Кол-во пользователей:610
Кол-во рейтингов:25209
Кол-во фильмов:5640


### Тестрование и выбор алгоритма для дальнейшего использования
####  SVD   алгоритм

In [73]:
%%time
algo_test = {} # Словарь в который будеем сохранять результаты тестов каждого алгоритма

start = time.time() ## точка отсчета времени

# алгоритм SVD
# random_state=my_seed где my_seed - произвольное число для инициализации генератора случайных чисел. Задается для повторяемости результатов.
# Если не задать random_state результаты на каждой итерации могут немного отличаться.
algo = SVD(random_state=my_seed)

# Обучаем алгоритм на наборе trainset и предсказываем рейтинги для набора testset
algo.fit(trainset)
predictions = algo.test(testset)

# Расчитываем RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
# Записываем результат в словарьы
algo_test['SVD'] = acc, end

RMSE: 0.8733
CPU times: user 1.21 s, sys: 13.7 ms, total: 1.22 s
Wall time: 1.23 s


In [75]:
print("Кол-во прогнозных рейтингов:", len(predictions))
predictions[0:5]

Кол-во прогнозных рейтингов: 25209


[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.167057749302103, details={'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.062812931475895, details={'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.0683809734741874, details={'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.156186391975082, details={'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.9878677739516248, details={'was_impossible': False})]

#### Пояснения:
В итоге мы спрогнозировали рейтинги по тестовому набру используя `алгоритм сингулярного разложения SVD`.
<pre>
Prediction(uid=376, iid=4896, r_ui=4.5, est=4.167057749302103, details={'was_impossible': False}
</pre>
где:
> uid=376 -  ID пользователя<br>
> iid=4896 - ID фильма<br>
> `r_ui=4.5` - Реальный рейтинг, который поставил пользователь<br>
> `est=4.167057749302103` - Расчетный рейтинг - рейтинг, который спрогнозировал алгоритм.<br>

Мы видим визуально по первым пяти строкам, что алгоритм довольно точно спрогннозировал метрики.

`RMSE: 0.8733` - на всем тестовом наборе среднеквадратичнная ошибка тоже относительно маленнькая, что говорит о хорошей точности алгоритма ("наивный метод показывал RMSE:  2.467566478745013")

#### SVDpp алгоритм

In [51]:
%%time
start = time.time() ## точка отсчета времени
# The SVD++ algorithm, an extension of SVD taking into account implicit ratings.
algo = SVDpp(random_state=my_seed)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['SVDpp'] = acc, end

RMSE: 0.8664
CPU times: user 1min 8s, sys: 222 ms, total: 1min 9s
Wall time: 1min 9s


In [52]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.3502238289679545, details={'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.068600030979475, details={'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.4250114836369883, details={'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.449656980140625, details={'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.663085639379482, details={'was_impossible': False})]

#### NormalPredictor алгоритм

In [53]:
%%time
start = time.time() ## точка отсчета времени
# Algorithm predicting a random rating based on the distribution of the training set, 
# which is assumed to be normal.
algo = NormalPredictor()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['NormalPredictor'] = acc,end

RMSE: 1.4290
CPU times: user 225 ms, sys: 14.2 ms, total: 239 ms
Wall time: 311 ms


In [54]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=3.5302376990668147, details={'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=3.7745225267048754, details={'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.0900907906666024, details={'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=3.289368675665574, details={'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=2.174872520089718, details={'was_impossible': False})]

#### BaselineOnly алгоритм

In [55]:
%%time
start = time.time() ## точка отсчета времени
#Algorithm predicting the baseline estimate for given user and item.
algo = BaselineOnly()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['BaselineOnly'] = acc,end

Estimating biases using als...
RMSE: 0.8740
CPU times: user 364 ms, sys: 2.8 ms, total: 366 ms
Wall time: 365 ms


In [56]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.179907225203624, details={'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.069041999109828, details={'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.2722360664098153, details={'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.364559826905557, details={'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.9893996392136961, details={'was_impossible': False})]

#### KNNBasic алгоритм

In [57]:
%%time
start = time.time() ## точка отсчета времени
#A basic collaborative filtering algorithm..
algo = KNNBasic()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['KNNBasic'] = acc,end

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9513
CPU times: user 1.56 s, sys: 13.4 ms, total: 1.58 s
Wall time: 1.58 s


In [58]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=3.8793347572869905, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.277201851301991, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.611485451182821, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.197985688843111, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=2.389003053309602, details={'actual_k': 27, 'was_impossible': False})]

#### KNNWithMeans алгоритм

In [59]:
%%time
start = time.time() ## точка отсчета времени
#A basic collaborative filtering algorithm, taking into account the mean ratings of each user.
algo = KNNWithMeans()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['KNNWithMeans'] = acc,end

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9020
CPU times: user 1.75 s, sys: 13.3 ms, total: 1.76 s
Wall time: 1.77 s


In [60]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.247008496480693, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.070840806833144, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.2534450574340097, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.611695414990949, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.7250530363280587, details={'actual_k': 27, 'was_impossible': False})]

#### KNNWithZScore алгоритм

In [61]:
%%time
start = time.time() ## точка отсчета времени
#A basic collaborative filtering algorithm, taking into account the z-score normalization of each user.
algo = KNNWithZScore()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['KNNWithZScore'] = acc,end

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9029
CPU times: user 2.04 s, sys: 12.5 ms, total: 2.06 s
Wall time: 2.07 s


In [62]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.2278695463491776, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.010113578939818, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.2520220054761855, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.506381667143983, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.8167762178190512, details={'actual_k': 27, 'was_impossible': False})]

#### KNNBaseline алгоритм

In [63]:
%%time
start = time.time() ## точка отсчета времени
# A basic collaborative filtering algorithm taking into account a baseline rating.
algo = KNNBaseline()
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['KNNBaseline'] = acc,end

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8788
CPU times: user 2.19 s, sys: 10.7 ms, total: 2.2 s
Wall time: 2.21 s


In [64]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.140442877662954, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.095896399572163, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.357475193469179, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.466303432323974, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.7388489629459953, details={'actual_k': 27, 'was_impossible': False})]

#### NMF алгоритм

In [65]:
%%time
start = time.time() ## точка отсчета времени
# A collaborative filtering algorithm based on Non-negative Matrix Factorization.
algo = NMF(random_state=my_seed)
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['NMF'] = acc,end

RMSE: 0.9292
CPU times: user 2.32 s, sys: 8.08 ms, total: 2.32 s
Wall time: 2.33 s


In [66]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.010190073923741, details={'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.065719646354205, details={'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.2435522304394158, details={'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.099059459077494, details={'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.9020361830112946, details={'was_impossible': False})]

#### SlopeOne алгоритм

In [67]:
%%time
start = time.time() ## точка отсчета времени
# A simple yet accurate collaborative filtering algorithm.
algo = SlopeOne()
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['SlopeOne'] = acc,end

RMSE: 0.9062
CPU times: user 9.79 s, sys: 400 ms, total: 10.2 s
Wall time: 10.2 s


In [68]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.145204041649962, details={'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=4.142859210390142, details={'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.445661245829139, details={'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.401728298701436, details={'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.5913794575502833, details={'was_impossible': False})]

#### CoClustering алгоритм

In [69]:
%%time
start = time.time() ## точка отсчета времени
# A collaborative filtering algorithm based on co-clustering.
algo = CoClustering(random_state=my_seed)
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
acc = accuracy.rmse(predictions)
end = time.time() - start ## время работы программы
algo_test['CoClustering'] = acc,end

RMSE: 0.9581
CPU times: user 2.08 s, sys: 115 ms, total: 2.19 s
Wall time: 2.18 s


In [70]:
predictions[0:5]

[Prediction(uid=376, iid=4896, r_ui=4.5, est=4.035735849623634, details={'was_impossible': False}),
 Prediction(uid=332, iid=4973, r_ui=4.0, est=3.8410505729358513, details={'was_impossible': False}),
 Prediction(uid=509, iid=112138, r_ui=3.0, est=3.153993978233908, details={'was_impossible': False}),
 Prediction(uid=221, iid=32, r_ui=5.0, est=4.618555366408149, details={'was_impossible': False}),
 Prediction(uid=599, iid=455, r_ui=2.0, est=1.419726108820588, details={'was_impossible': False})]

In [71]:
# Вывод результатов точности предсказания алгоритмов
algo_test = dict(sorted(algo_test.items(), key=lambda item: item[1]))
print("Сводный отчет по точности и времеи прогнозированния алгоритмов (сортировка по возрастаниню RMSE):")
algo_test

Сводный отчет по точности и времеи прогнозированния алгоритмов (сортировка по возрастаниню RMSE):


{'SVDpp': (0.8664002776978356, 69.1892900466919),
 'SVD': (0.8732855533215714, 1.1636848449707031),
 'BaselineOnly': (0.8739810025477771, 0.36529111862182617),
 'KNNBaseline': (0.8787968697683455, 2.2062129974365234),
 'KNNWithMeans': (0.9020168225607583, 1.77052903175354),
 'KNNWithZScore': (0.9028537907713149, 2.065805196762085),
 'SlopeOne': (0.9062262055862815, 10.179553031921387),
 'NMF': (0.9292244337972277, 2.329216957092285),
 'KNNBasic': (0.9512862693475453, 1.5840981006622314),
 'CoClustering': (0.9580609818718936, 2.1825878620147705),
 'NormalPredictor': (1.429021476394733, 0.3106660842895508)}

### Выбор алгоритма для дальнейго использования: 
SVDpp - самый точный, но очень медленный.

**SVD - один из самых быстрых и точных.** Этот алгоритм будем использовать для получение топ-N рекомендаций для пользователя.

BaselineOnly - один из самых быстрых, по точности немного уступает SVD

**KNNBaseline - по точности и скорости немного уступает BaselineOnly и SVD.** Этот алгоритм будем использовать для получение 10 ближайших фильмов.

Алгортмы семейства kNN удобно использовать для получения k-ближайших соседей пользователя (или элемента).
Можно использовать методы `get_neighbors()` объекта алгоритма. Это актуально только для алгоритмов, использующих меру сходства, таких как алгоритмы k-NN.

### Получение топ-N рекомендаций для каждого пользователя

Мы извлекаем 10 лучших элементов с самым высоким прогнозом рейтинга для каждого пользователя в наборе данных MovieLens. 
Сначала мы обучаем алгоритм SVD на всем наборе данных, а затем прогнозируем все нулевые рейтинги для пар (пользователь, элемент). Затем мы получаем прогноз топ-10 для каждого пользователя.

In [76]:
def get_top_n(predictions, n=10):
    """Возвращает первые N рекомендаций для каждого пользователя из набора прогнозов.

    Args:
        predictions(список предсказанных объектов): Список предсказаний,
             возвращаемый алгоритмом.
         n(int): Количество рекомендаций для вывода для каждого пользователя. По умолчанию
             это 10.

    Returns:
    Список, где ключи — это идентификаторы пользователей, а значения — это списки кортежей c 
    идентификатором фильма и расчетным рейтингом:
         [userId: [(movieId, rating_est),(movieId, rating_est),.....],...] размера n
    """

    # Сначала сопоставьте прогнозы с каждым пользователем.
    # Используя функцию list() в качестве функции в defaultdict, легко сгруппировать 
    # последовательность кортежей ключ-значение в словарь списков
    top_n = defaultdict(list) 
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Затем отсортируем прогнозы для каждого пользователя и извлечем k самых высоких.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [77]:
# Функция вывода результатов рекомендаций
def movie_recommend(UserID):
    # Просмотреные пользователем фильмы
    viewed_films = pd.merge(df_ratings[df_ratings['userId'] == UserID], df_movies, on='movieId')

    # Рекомендованные фильмы
    recommend = pd.DataFrame({'Move ID':[],'Title':[], 'Genres':[], 'Rating estimated':[]})
    for moveID, RATING in top_n.get(UserID):
        idx = len(recommend.index)
        film = list(df_movies[df_movies['movieId'] == moveID].values[0])
        film.append(RATING)
        recommend.loc[idx] = film
    
    return viewed_films, recommend

In [78]:
%%time
# Сначала обучаем алгоритм SVD на наборе данных movielens.
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df_ratings[["userId", "movieId", "rating"]], reader)
trainset = data.build_full_trainset() # Вся выборка из df_ratings
algo = SVD(random_state=my_seed)
algo.fit(trainset) # Обучение алгоритма

# build_anti_testset() - создает набор с рейтингами, которых нет в составе тренировочного набора (trainset), т.е.
# все рейтинги с "0" или "NaN", где известен пользователь, известен элемент, но рейтинг 
# отсутствует в тренировочной выборке (trainset). 
# Поскольку рейтинг r_ui (реальный) не известен, он либо заменяется на значение fill=None 
# или предполагается равным среднему значению всех оценок.
testset = trainset.build_anti_testset()
# Затем предсказываем оценки для всех пар (u, i), которые НЕ проставлены пользователем (равны 0).
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10) # n=10 - кол-во рекомендаций

CPU times: user 55 s, sys: 1.6 s, total: 56.6 s
Wall time: 56.8 s


In [79]:
# Расчитываем RMSE
# RMSE должен быть низким, так как мы предвзяты
accuracy.rmse(predictions, verbose=True)

RMSE: 0.4855


0.4854911986827064

In [80]:
print("Кол-во метрик:", len(predictions))
predictions[0:5]

Кол-во метрик: 5830804


[Prediction(uid=1, iid=318, r_ui=3.501556983616962, est=5.0, details={'was_impossible': False}),
 Prediction(uid=1, iid=1704, r_ui=3.501556983616962, est=4.826856301266907, details={'was_impossible': False}),
 Prediction(uid=1, iid=6874, r_ui=3.501556983616962, est=4.844444062542448, details={'was_impossible': False}),
 Prediction(uid=1, iid=8798, r_ui=3.501556983616962, est=4.511979880202485, details={'was_impossible': False}),
 Prediction(uid=1, iid=46970, r_ui=3.501556983616962, est=4.343640761208876, details={'was_impossible': False})]

#### Пояснения:
В итоге в predictions у нас храннятся все прогнозные значения рейитнгов `est` всех пользователей по кажддому фильму. Т.е. везде где в рейтингах были "0" теперь рейтинги от 0.000000 до 5.0

In [81]:
results_df = pd.DataFrame.from_dict(predictions)
results_df

,uid,iid,r_ui,est,details
0,1,318,3.501557,5.000000,{'was_impossible': False}
1,1,1704,3.501557,4.826856,{'was_impossible': False}
2,1,6874,3.501557,4.844444,{'was_impossible': False}
3,1,8798,3.501557,4.511980,{'was_impossible': False}
4,1,46970,3.501557,4.343641,{'was_impossible': False}
...,...,...,...,...,...
5830799,610,7377,3.501557,3.641296,{'was_impossible': False}
5830800,610,8667,3.501557,3.522076,{'was_impossible': False}
5830801,610,32302,3.501557,3.389062,{'was_impossible': False}
5830802,610,51903,3.501557,3.376642,{'was_impossible': False}


In [82]:
# Пример вывода расчетных рейтингов всех пользователей для фильма с ID 110, которые не смотрели этот фильм.
results_df[results_df['iid']==110][['uid', 'iid', 'est']]

,uid,iid,est
9499,2,110,4.183066
19194,3,110,2.736599
28878,4,110,3.038161
57475,7,110,3.588752
76726,9,110,3.813461
...,...,...,...
5645922,591,110,3.748657
5684334,595,110,4.484273
5694037,596,110,3.858135
5712633,598,110,4.134596


In [83]:
# Вывод рекомендаций для пользователя UserID
UserID = 126
viewed_films, recommend = movie_recommend(UserID)

print(f"Просмотренные фильмы пользователя {UserID}:")
viewed_films

Просмотренные фильмы пользователя 126:


,userId,movieId,rating,title,genres
0,126,34,3.0,Babe (1995),Children|Drama
1,126,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
2,126,110,4.0,Braveheart (1995),Action|Drama|War
3,126,150,4.0,Apollo 13 (1995),Adventure|Drama|IMAX
4,126,153,4.0,Batman Forever (1995),Action|Adventure|Comedy|Crime
5,126,161,3.0,Crimson Tide (1995),Drama|Thriller|War
6,126,165,4.0,Die Hard: With a Vengeance (1995),Action|Crime|Thriller
7,126,185,3.0,"Net, The (1995)",Action|Crime|Thriller
8,126,208,2.0,Waterworld (1995),Action|Adventure|Sci-Fi
9,126,231,1.0,Dumb & Dumber (Dumb and Dumber) (1994),Adventure|Comedy


In [84]:
print(f"Рекомендация фильмов для пользователя {UserID}:")
recommend

Рекомендация фильмов для пользователя 126:


,Move ID,Title,Genres,Rating estimated
0,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,4.163654
1,1172,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.157907
2,1104,"Streetcar Named Desire, A (1951)",Drama,4.154087
3,34405,Serenity (2005),Action|Adventure|Sci-Fi,4.132483
4,1276,Cool Hand Luke (1967),Drama,4.126663
5,720,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy,4.121370
6,1213,Goodfellas (1990),Crime|Drama,4.115909
7,3275,"Boondock Saints, The (2000)",Action|Crime|Drama|Thriller,4.099605
8,1223,"Grand Day Out with Wallace and Gromit, A (1989)",Adventure|Animation|Children|Comedy|Sci-Fi,4.081278
9,111,Taxi Driver (1976),Crime|Drama|Thriller,4.076663


### Получение k ближайших соседей пользователя (или элемента)
Можно использовать методы get_neighbors() объекта алгоритма. Это актуально только для алгоритмов, использующих меру сходства, таких как алгоритмы k-NN.

Мы извлекаем 10 ближайших соседей заданного фильма из набора данных MovieLens.

In [85]:
%%time
# Обучаем алгоритм вычислять сходство между элементами.
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df_ratings[["userId", "movieId", "rating"]], reader)

trainset = data.build_full_trainset()
sim_options = {
    "name": "pearson_baseline", # Вычислить коэффициент корреляции Пирсона между всеми парами пользователей (или элементов).
    "user_based": False # Вычислить сходство между элементами (если True - то между пользователями).
}

algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
CPU times: user 5.14 s, sys: 1.77 s, total: 6.91 s
Wall time: 7 s


In [86]:
# Функция рекомендации для фильмов
def item_recommend(FilmID):
    # Получить внутренний идентификатор фильма
    move_inner_id = algo.trainset.to_inner_iid(FilmID)
    #print("move_inner_id:",move_inner_id)

    # Получить внутренние идентификаторы ближайших соседей фильма.
    move_neighbors = algo.get_neighbors(move_inner_id, k=10)
    #print("move_neighbors:", move_neighbors)

    # Преобразование внутренних идентификаторов соседей в идентификаторы фильмов исходных данных (df_movies).
    move_neighbors = (
        algo.trainset.to_raw_iid(inner_id) for inner_id in move_neighbors
    )
    
    # Вывод названия фильма по его ID
    for_film = df_movies[df_movies['movieId']==FilmID]
    
    # Вывод рекомендованных фильмов
    recommend_films = pd.DataFrame({'movieId':[],'title':[], 'genres':[]})
    for moveID in move_neighbors:
        #print(moveID)
        idx = len(recommend_films.index)
        film = list(df_movies[df_movies['movieId'] == moveID].values[0])
        recommend_films.loc[idx] = film

    return for_film, recommend_films

In [87]:
# Рекомендация для фильма FilmID
FilmID =2628
for_film, recommend_films = item_recommend(FilmID)

print("Пользователям, которым нравится фильм:")
print(for_film)
print("\nрекомендуется к просмотру:")
recommend_films

Пользователям, которым нравится фильм:
      movieId                                             title  \
1979     2628  Star Wars: Episode I - The Phantom Menace (1999)   

                       genres  
1979  Action|Adventure|Sci-Fi  

рекомендуется к просмотру:


,movieId,title,genres
0,5378,Star Wars: Episode II - Attack of the Clones (...,Action|Adventure|Sci-Fi|IMAX
1,33493,Star Wars: Episode III - Revenge of the Sith (...,Action|Adventure|Sci-Fi
2,6934,"Matrix Revolutions, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
3,6365,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
4,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
5,380,True Lies (1994),Action|Adventure|Comedy|Romance|Thriller
6,1917,Armageddon (1998),Action|Romance|Sci-Fi|Thriller
7,1876,Deep Impact (1998),Drama|Sci-Fi|Thriller
8,1784,As Good as It Gets (1997),Comedy|Drama|Romance
9,2706,American Pie (1999),Comedy|Romance
